<a href="https://colab.research.google.com/gist/lrtipple/d8fa0354e3c764b8d999251a6554e4ca/housepricesdatabasedataload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [2]:
# use spark to write to postgres database
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-04 22:52:26--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.91MB/s    in 0.2s    

2022-07-04 22:52:26 (4.91 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
# start a spark session with option to add driver to Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()


In [7]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://housepricespub.s3.us-east-2.amazonaws.com/train.csv"
spark.sparkContext.addFile(url)
house_data_df = spark.read.csv(SparkFiles.get("train.csv"), sep=",", header=True, inferSchema=True)

In [ ]:
# Show DataFrame
house_data_df.show()

In [6]:
url ="https://housepricespub.s3.us-east-2.amazonaws.com/prices.csv"
spark.sparkContext.addFile(url)
house_prices_df = spark.read.csv(SparkFiles.get("prices.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
house_prices_df.show()

+---+---------+
| Id|SalePrice|
+---+---------+
|  1|   208500|
|  2|   181500|
|  3|   223500|
|  4|   140000|
|  5|   250000|
|  6|   143000|
|  7|   307000|
|  8|   200000|
|  9|   129900|
| 10|   118000|
| 11|   129500|
| 12|   345000|
| 13|   144000|
| 14|   279500|
| 15|   157000|
| 16|   132000|
| 17|   149000|
| 18|    90000|
| 19|   159000|
| 20|   139000|
+---+---------+
only showing top 20 rows



In [14]:
# Store environmental variable
from getpass import getpass
password = getpass('finaldal')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://houseprices11.cbchqixwjhxz.us-east-2.rds.amazonaws.com:5432/houseprices"
config = {"user":"postgres",
          "password": password,  
          "driver":"org.postgresql.Driver"}

finaldal··········


In [17]:
# Write DataFrame to house data table in RDS
house_data_df.write.jdbc(url=jdbc_url, table='houses', mode=mode, properties=config)

In [18]:
# Write dataframe to price table in RDS
house_prices_df.write.jdbc(url=jdbc_url, table='prices', mode=mode, properties=config)

In [20]:
# join tables
joined_df = house_data_df.join(house_prices_df, on="Id", how="left")
joined_df

DataFrame[Id: int, MSSubClass: int, MSZoning: string, LotFrontage: string, LotArea: int, Street: string, Alley: string, LotShape: string, LandContour: string, Utilities: string, LotConfig: string, LandSlope: string, Neighborhood: string, Condition1: string, Condition2: string, BldgType: string, HouseStyle: string, OverallQual: int, OverallCond: int, YearBuilt: int, YearRemodAdd: int, RoofStyle: string, RoofMatl: string, Exterior1st: string, Exterior2nd: string, MasVnrType: string, MasVnrArea: string, ExterQual: string, ExterCond: string, Foundation: string, BsmtQual: string, BsmtCond: string, BsmtExposure: string, BsmtFinType1: string, BsmtFinSF1: int, BsmtFinType2: string, BsmtFinSF2: int, BsmtUnfSF: int, TotalBsmtSF: int, Heating: string, HeatingQC: string, CentralAir: string, Electrical: string, 1stFlrSF: int, 2ndFlrSF: int, LowQualFinSF: int, GrLivArea: int, BsmtFullBath: int, BsmtHalfBath: int, FullBath: int, HalfBath: int, BedroomAbvGr: int, KitchenAbvGr: int, KitchenQual: string

In [21]:
# Write dataframe to price table in RDS
joined_df.write.jdbc(url=jdbc_url, table='joined_data', mode=mode, properties=config)